# HW3

Submit via Slack. Due on **Tuesday, April 12th, 2022, 6:29pm PST**. You may work with one other person.
## TF-IDF (5pts)

You are an analyst working for Amazon's product team, and charged with identifying areas for improvement for the toy reviews.

Using the **amazon-fine-foods.csv** dataset, clean and parse the text reviews. Explain the decisions you make:
- why remove/keep stopwords?
- which stopwords to remove?
- stemming versus lemmatization?
- regex cleaning and substitution?
- adding in custom stopwords?
- what `n` for your `n-grams`?
- which words to collocate together?

Finally, generate a TF-IDF report that explains for a business (non-technical) stakeholder:
* the features your analysis showed that customers cited as reasons for a poor review
* the features your analysis showed that customers cited as reasons for a good review
* the most common issues identified from your analysis that generated customer dissatisfaction.

Explain to what degree the TF-IDF findings make sense - what are its limitations?


In [8]:
import re
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from textacy import preprocessing

In [3]:
df = pd.read_csv('../datasets/amazon_fine_foods.csv')

In [4]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,20983,B002QWP89S,A21U4DR8M6I9QN,"K. M Merrill ""justine""",1,1,5,1318896000,addictive! but works for night coughing in dogs,my 12 year old sheltie has chronic brochotitis...
1,20984,B002QWP89S,A17TDUBB4Z1PEC,jaded_green,1,1,5,1318550400,genuine Greenies best price,"These are genuine Greenies product, not a knoc..."
2,20985,B002QWP89S,ABQH3WAWMSMBH,tenisbrat87,1,1,5,1317168000,Perfect for our little doggies,"Our dogs love Greenies, but of course, which d..."
3,20986,B002QWP89S,AVTY5M74VA1BJ,tarotqueen,1,1,5,1316822400,dogs love greenies,"What can I say, dogs love greenies. They begg ..."
4,20987,B002QWP89S,A13TNN54ZEAUB1,dcz2221,1,1,5,1316736000,Greenies review,This review is for a box of Greenies Lite for ...


In [ ]:
# chips, sweets, coconut oil
df = df[(df['ProductId'] == 'B006HYLW32') | (df['ProductId'] == 'B005K4Q1YA') | (df['ProductId'] == 'B001EO5Q64')]

In [16]:
# using textacy to: remove hyphens, punctuation, and accents
preproc = preprocessing.make_pipeline(
    preprocessing.remove.html_tags,
    preprocessing.normalize.hyphenated_words,
    preprocessing.remove.punctuation,
    preprocessing.remove.accents,
    

)

In [12]:
set(stopwords.words('english'))
# removing some negative words from stopwords list
nltk_stopwords = set(stopwords.words('english'))
nltk_stopwords.remove('below')
nltk_stopwords.remove("aren't")
nltk_stopwords.remove('couldn')
nltk_stopwords.remove("couldn't")
nltk_stopwords.remove("didn't")

While it may be easier to use the pre-built function, it does not allow for changing the stopwords list. Therefore I am creating the function manually.

In [15]:
# removing stopwords 
def remove_stopwords(sentence:str, nltk_stopwords):
    '''removing stopwords from a list, review: string, nltk_stopwords: list'''
    words = nltk.word_tokenize(sentence)
    new_words = []
    for word in words:
        if word.lower() in nltk_stopwords:
            continue
        new_words.append(word)
    cleaned_review = " ".join(new_words)
    return cleaned_review

In [17]:
df['text_cleaned'] = df['Text'].apply(preproc).apply(remove_stopwords, nltk_stopwords=nltk_stopwords)
df.head()


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,text_cleaned
0,20983,B002QWP89S,A21U4DR8M6I9QN,"K. M Merrill ""justine""",1,1,5,1318896000,addictive! but works for night coughing in dogs,my 12 year old sheltie has chronic brochotitis...,12 year old sheltie chronic brochotitis meds t...
1,20984,B002QWP89S,A17TDUBB4Z1PEC,jaded_green,1,1,5,1318550400,genuine Greenies best price,"These are genuine Greenies product, not a knoc...",genuine Greenies product knockoff dogs love fa...
2,20985,B002QWP89S,ABQH3WAWMSMBH,tenisbrat87,1,1,5,1317168000,Perfect for our little doggies,"Our dogs love Greenies, but of course, which d...",dogs love Greenies course doggies bought dashc...
3,20986,B002QWP89S,AVTY5M74VA1BJ,tarotqueen,1,1,5,1316822400,dogs love greenies,"What can I say, dogs love greenies. They begg ...",say dogs love greenies begg time always sit cu...
4,20987,B002QWP89S,A13TNN54ZEAUB1,dcz2221,1,1,5,1316736000,Greenies review,This review is for a box of Greenies Lite for ...,review box Greenies Lite dog package came quic...


In [ ]:
# since the corpus is food reviews, we use regex to group certain words or phrases that are related to certain tastes.
# good tastes
df['text_cleaned'] = df['text_cleaned'].str.replace(r'\bdelicious|tasty|yum+y*\b', '_GOOD_TASTE_', case=False)

In [ ]:
# sweet
df['text_cleaned'] = df['text_cleaned'].str.replace(r'\bsugar(y|ed)?|swe{2,}t(en(ed)?)?\b', '_SWEET_', case=False)

In [ ]:
# savory
df['text_cleaned'] = df['text_cleaned'].str.replace(r'\bsalty*|savou?ry\b', '_SAVORY_', case=False)

In [ ]:
# removing all words with more than 15 digits
df['text_cleaned'] = df['text_cleaned'].str.replace(r'\b([a-z]|[A-Z]){15,}\b', '', case=False)

In [ ]:
# bad tastes
df['text_cleaned'] = df['text_cleaned'].str.replace(r'\bdisgusting|ugh\b', '_GOOD_TASTE_', case=False)

* stemming/lemmatization (explain in your notebook why you used stemming versus lemmatization). 

Same as HW2, we choose lemmatization since it can better account for transformations that are not standard. Such transformations can be quite common in reviews. For example, it is very likely for reviews to contain `better` or `worse`, neither of which can be treated with stemming. Moreover, we do not have a performance limitation as the dataset is rather small and we are not doing realtime transformation.

In [ ]:
lemmatizer = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)


In [ ]:
df['text_cleaned'] = df['text_cleaned'].apply(lemmatize_sentence)

- Given the rather small dataset, we find trigrams too specific to give meaning results. Therefore, we choose 2 (bigram).

In [ ]:
#

## Similarity and Word Embeddings (2 pts)

Using
* `TfIdfVectorizer`

Identify the most similar pair of reviews from the `amazon-fine-foods.csv` dataset using both Euclidean distance and cosine similarity.

## Naive Bayes (3pts)

You are an NLP data scientist working at Fandango. You observe the following dataset in your review comments:

**Intent to Buy Tickets:**
1.	Love this movie. Can’t wait!
2.	I want to see this movie so bad.
3.	This movie looks amazing.

**No Intent to Buy Tickets:**
1.	Looks bad.
2.	Hard pass to see this bad movie.
3.	So boring!

You can consider the following stopwords for removal: `to`, `this`.

Is the following review an `Intent to Buy` or `No Intent to Buy`? Show your work for each computation.
> This looks so bad.

You'll need to compute:
* Prior
* Likelihood
* Posterior